In [1]:
import pandas as pd
import numpy as np
from numpy import abs
from numpy import log
from numpy import sign
from datetime import date
from numba import njit 
import seaborn as sns
from scipy.stats import spearmanr

In [2]:
df = pd.read_pickle('data/merged_data_log60.pkl')
df

,date,ticker,open,high,low,close,volume,returns,report_date,AccountsPayable_x,...,ProceedsFromLongTermDebt,PropertyAndPlantAndEquipment,RealizedGain_y,ReceivableIncrease,RedemptionOfBonds,RentalPrincipalRepayments,RepaymentOfLongTermDebt,TotalIncomeLossItems,UnrealizedGain_y,effective_date
0,2015-01-05,1101.TW,19.371103,19.371103,19.103300,19.192568,3847400.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,2015-01-05,1612.TW,5.573573,5.627949,5.546384,5.573573,140000.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2015-01-05,6128.TW,16.387497,16.555698,16.291382,16.387497,53078.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,2015-01-05,1611.TW,8.408819,8.564537,8.408819,8.564537,224370.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,2015-01-05,6133.TW,8.562571,8.768402,8.562571,8.644904,142000.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503072,2025-11-24,2457.TW,28.950001,28.950001,27.700001,27.799999,3172422.0,-0.033044,2025-06-30,1.813032e+09,...,1.500000e+09,-2.290790e+08,NaN,308406000.0,0.0,-23745000.0,-7.433000e+06,1.382110e+08,NaN,2025-08-29
2503073,2025-11-24,3533.TW,1180.000000,1185.000000,1140.000000,1165.000000,1075453.0,0.008658,2025-06-30,2.795921e+09,...,0.000000e+00,-2.408501e+09,NaN,815614000.0,NaN,-74868000.0,0.000000e+00,1.106427e+09,NaN,2025-08-29
2503074,2025-11-24,6863.TW,110.000000,111.000000,109.500000,111.000000,65074.0,0.027778,2025-06-30,3.619960e+08,...,NaN,-2.987520e+08,NaN,303761000.0,NaN,-6175000.0,NaN,1.010770e+08,NaN,2025-08-29
2503075,2025-11-24,2303.TW,45.450001,46.549999,45.000000,46.549999,96809729.0,0.034444,2025-06-30,8.538152e+09,...,2.800000e+09,-2.169565e+10,NaN,-77444000.0,0.0,-410117000.0,-1.759375e+10,2.608948e+10,NaN,2025-08-29


In [3]:
# Alpha 101 基礎運算函式庫與資料準備
# 基礎欄位
open = df["open"]
high = df['high']
low = df['low']
close = df['close']
volume = df['volume']
returns = df['returns']
vwap_daily_volume = df.groupby('date')['volume'].transform('sum')
vwap = (close * volume).groupby(df['date']).transform('sum') / vwap_daily_volume.replace(0, np.nan)

# 輔助函式
def vmin(a, b):
    if np.isscalar(b):                  # b 是 integer → rolling min
        return ts_min(a, b) 
    else:                               # b 是 series → elementwise min
        return np.minimum(a, b)
# 輔助函式
def vmax(a, b):
    if isinstance(b, int) and b > 0:              # b 是 integer → rolling min
        return ts_max(a, b)
    else:                               # b 是 series → elementwise min
        return np.maximum(a, b)
    
def ts_sum(s,d):
    return s.groupby(df['ticker']).rolling(d, min_periods=1).sum().reset_index(0,drop=True)

def sma(s, d):
    return s.groupby(df['ticker']).rolling(d,min_periods=1).mean().reset_index(0, drop=True)

def stddev(s, d):
    return s.groupby(df['ticker']).rolling(d, min_periods=1).std().reset_index(0, drop=True)

def correlation(x,y,d):
    def rolling_corr(group):
        return group['x'].rolling(d, min_periods=d//2).corr(group['y'])
    temp_df = pd.DataFrame({'x': x, 'y': y, 'ticker': df['ticker']})
    corr_series = temp_df.groupby('ticker').apply(rolling_corr).reset_index(level=0, drop=True)
    return corr_series

def covariance(x, y, d):
    def rolling_cov(group):
        return group['x'].rolling(d, min_periods=d//2).cov(group['y'])

    temp_df = pd.DataFrame({'x': x, 'y': y, 'ticker': df['ticker']})
    cov_series = temp_df.groupby('ticker').apply(rolling_cov).reset_index(level=0, drop=True)
    return cov_series

def delay(s,d):
    return s.groupby(df['ticker']).shift(d)

def delta(s,d):
    return s.groupby(df['ticker']).diff(d)

def cs_rank(s):
    return s.groupby(df['date']).rank(pct=True)

def scale(s,a=1):
    sum_abs = s.groupby(df['date']).transform(lambda x: x.abs().sum())
    return s * a / sum_abs.replace(0,1)

def ts_max(s,d):
    return s.groupby(df['ticker']).rolling(d, min_periods=1).max().reset_index(0,drop=True)

def ts_min(s,d):
    return s.groupby(df['ticker']).rolling(d, min_periods=1).min().reset_index(0,drop=True)

def ts_argmax(s,d):
    return s.groupby(df['ticker']).rolling(d, min_periods=1).apply(np.argmax, raw=True).reset_index(0, drop=True) + 1

def ts_argmin(s,d):
    return s.groupby(df['ticker']).rolling(d, min_periods=1).apply(np.argmin, raw=True).reset_index(0, drop=True) + 1

def ts_rank(s,d):
    def rank_pct(x):
        return pd.Series(x).rank(pct=True).iloc[-1]
    return s.groupby(df['ticker']).rolling(d, min_periods=d).apply(rank_pct, raw=False).reset_index(0, drop=True)

def signed_power(s, a):
    return sign(s) * (abs(s) ** a)

# 這個暫時沒用到
def product(x, d):
    return x.rolling(d).apply(lambda s: s.prod(), raw=True)

def ts_product(s, d):
    return s.groupby(df['ticker']).rolling(d, min_periods=1).apply(np.prod, raw=True).reset_index(0, drop=True)

def ts_decay_linear(s, d):
     def weighted_average(x):
        weights = np.arange(1, len(x) + 1)
        return np.dot(x, weights) / weights.sum()
     return s.groupby(df['ticker']).rolling(d, min_periods=d//2).apply(weighted_average, raw=True).reset_index(0, drop=True)

def adv(vwap, volume, d):
    dv = vwap * volume                          # dollar volume
    return dv.groupby(df['ticker']) \
            .rolling(int(d), min_periods=1) \
            .mean() \
            .reset_index(0, drop=True)

# Alpha 158 基礎運算函式庫與資料準備

def ts_slope(s, d):
    idx = np.arange(d)

    return (
        s.groupby(df['ticker'])
         .rolling(d, min_periods=d)                      # 回歸需要完整資料
         .apply(lambda x: np.polyfit(idx, x, 1)[0], raw=True)
         .reset_index(level=0, drop=True)
    )

def ts_rsq(s, n):
    def _rsq(x):
        # x: numpy array for that rolling window
        x = np.asarray(x, float)

        # 處理 NaN，避免 polyfit 爆掉
        mask = ~np.isnan(x)
        x = x[mask]
        m = len(x)
        if m < 2:
            return np.nan   # 點太少不回歸，回傳 NaN

        # 時間 index 依照實際有效長度來建
        idx = np.arange(m)

        # 拟合回归：x ≈ a + b * t
        b, a = np.polyfit(idx, x, 1)
        y_hat = a + b * idx

        # R-squared = 1 - SSE / SST
        ss_res = ((x - y_hat) ** 2).sum()
        ss_tot = ((x - x.mean()) ** 2).sum()
        return 1 - ss_res / ss_tot if ss_tot != 0 else 0

    return (
        s.groupby(df['ticker'])       # 和你原本一樣用全域 df['ticker'] 分組
         .rolling(n, min_periods=n)
         .apply(_rsq, raw=True)
         .reset_index(level=0, drop=True)
    )

def ts_resi(s, n):
    """
    s: df['close'] 這種 Series（含多檔股票）
    n: 回溯天數 N
    回傳：每個 ticker、每一天的『當前收盤價相對該 N 日趨勢線的殘差』
    """
    idx = np.arange(n)

    def _resi(x):
        # x 是長度 n 的 ndarray
        b, a = np.polyfit(idx, x, 1)           # y = a + b x
        y_hat = a + b * idx
        return x[-1] - y_hat[-1]              # 只取當下那點的殘差

    return (
        s.groupby(df['ticker'])
         .rolling(n, min_periods=n)           # 回歸需要 n 個點
         .apply(_resi, raw=True)
         .reset_index(level=0, drop=True)
    )

def ts_Quantile(s, d, q):
    """
    s: Series，例如 df['close']
    d: 過去 N 天的視窗長度
    q: 分位數，例如 0.8
    """
    return (
        s.groupby(df['ticker'])
         .rolling(d, min_periods=1)
         .quantile(q)
         .reset_index(0, drop=True)
    )

def IdxMax(s, d):
    # s 就是 high、low、close 這種 Series
    return (
        s.groupby(df['ticker'])
         .rolling(d, min_periods=1)
         .apply(lambda x: (len(x) - 1) - np.argmax(x.values), raw=False)
         .reset_index(0, drop=True)
    )

def IdxMin(s, d):
    # s 就是 high、low、close 這種 Series
    return (
        s.groupby(df['ticker'])
         .rolling(d, min_periods=1)
         .apply(lambda x: (len(x) - 1) - np.argmin(x.values), raw=False)
         .reset_index(0, drop=True)
    )

In [4]:
alpha101_expressions = {
    'alpha_001': 'cs_rank(ts_argmax(signed_power(close.where(returns >= 0, stddev(returns, 20)), 2.0), 5))',
    'alpha_002': '-1 * correlation(cs_rank(delta(np.log1p(volume), 2)), cs_rank((close - open) / open), 6)',
    'alpha_003': '-1 * correlation(cs_rank(open), cs_rank(volume),10)',
    'alpha_004': '-1 * ts_rank(cs_rank(low), 9)',
    'alpha_005': 'cs_rank(open - ts_sum(vwap,10)/10) * (-1 * abs(cs_rank(close - vwap)))',
    'alpha_006': '-1 * correlation(open, volume, 10)',
    'alpha_007': '-1 * ts_rank(abs(delta(close, 7)), 60) * sign(delta(close, 7)).where(adv(vwap,volume,20) < volume, -1)',
    'alpha_008': '-1 * cs_rank(ts_sum(open, 5) * ts_sum(returns, 5)- delay((ts_sum(open,5) * ts_sum(returns,5)),10))',
    'alpha_009': 'delta(close, 1).where(0 < ts_min(delta(close, 1), 5),delta(close, 1).where(ts_max(delta(close, 1), 5)<0,-1 * delta(close, 1)))',
    'alpha_010': 'cs_rank(delta(close,1).where(0 < ts_min(delta(close,1),4),delta(close,1).where(ts_max(delta(close,1),4) < 0,-1 * delta(close,1))))',
    'alpha_011': '(cs_rank(ts_max((vwap - close),3)) + cs_rank(ts_min((vwap - close),3))) * cs_rank(delta(volume,3))',
    'alpha_012': 'sign(delta(volume,1)) * (-1 * delta(close,1))',
    'alpha_013': '-1 * cs_rank(covariance(cs_rank(close), cs_rank(volume),5))',
    'alpha_014': '-1 * cs_rank(delta(returns, 3)) * correlation(open, volume,10)',
    'alpha_015': '-1 * ts_sum(cs_rank(correlation(cs_rank(high), cs_rank(volume),3)),3)',
    'alpha_016': '-1 * cs_rank(covariance(cs_rank(high), cs_rank(volume),5))',
    'alpha_017': '(((-1 * cs_rank(ts_rank(close,10))) * cs_rank(delta(delta(close, 1),1))) * cs_rank(ts_rank((volume/adv(vwap,volume,20)),5)))',
    'alpha_018': '-1 * cs_rank(((stddev(abs(close - open),5))+ (close - open)) + correlation(close, open, 10))',
    'alpha_019': '((-1 * sign((close - delay(close,7)) + delta(close,7))) * (1+ cs_rank(1+ts_sum(returns, 250))))',
    'alpha_020': '(((-1 * cs_rank(open-delay(high,1))) * cs_rank(open - delay(close, 1))) * cs_rank(open - delay(low,1)))',
    'alpha_021': '(close*0 - 1).where(((ts_sum(close, 8)/8) + stddev(close, 8)) < (ts_sum(close, 2)/2),(close*0 + 1).where((ts_sum(close, 2)/2) < ((ts_sum(close, 8)/8) - stddev(close, 8)),(close*0 + 1).where((volume / adv(vwap,volume,20)) >= 1,(close*0 - 1))))',
    'alpha_022': '(-1 * delta(correlation(high, volume,5),5) * cs_rank(stddev(close,20)))',
    'alpha_023': '(-1 * delta(high,2)).where(ts_sum(high, 20)/20 < high,0)',
    'alpha_024': '(-1*(close - ts_min(close,100))).where(delta(ts_sum(close,100)/100,100)/delay(close,100) <= 0.05, -1*delta(close,3))',
    'alpha_025': 'cs_rank(((-1*returns)*adv(vwap,volume,20)*vwap) * (high-close))',
    'alpha_026': '-1*ts_max(correlation(ts_rank(volume,5),ts_rank(high,5),5),3)',
    # 027 目前分配很怪，大多都是1，但理論上應該是-1和1各半
    'alpha_027': '(close*0-1).where(cs_rank(ts_sum(correlation(cs_rank(volume), cs_rank(vwap), 6), 2) / 2.0) > 0.5,close*0+1)',
    'alpha_028': 'scale(((correlation(adv(vwap,volume,20), low, 5) + ((high + low) / 2)) - close))',
    'alpha_029': '(ts_min(ts_product(cs_rank(cs_rank(scale(np.log1p(ts_sum(ts_min(cs_rank(cs_rank(-1 * cs_rank(delta(close- 1, 5)))), 2), 1))))), 1), 5) + ts_rank(delay(-1 * returns, 6), 5))',
    'alpha_030': '(((1.0 - cs_rank(sign(close - delay(close, 1)) + sign(delay(close, 1) - delay(close, 2)) + sign(delay(close, 2) - delay(close, 3)))) * ts_sum(volume, 5)) / ts_sum(volume, 20))',
    'alpha_031': 'cs_rank(cs_rank(cs_rank(ts_decay_linear(cs_rank(-1 * cs_rank(delta(close, 10))), 10)))) + cs_rank(-1 * delta(close, 3)+ sign(scale(correlation(adv(vwap,volume,20), low, 12))))',
    'alpha_033': 'cs_rank((-1*(1-(open/close))**1))',
    'alpha_034': 'cs_rank(1 - cs_rank(stddev(returns,2)/stddev(returns,5)) + (1-cs_rank(delta(close,1))))',
    'alpha_035': '((ts_rank(volume, 32)* (1 - ts_rank((close+high)-low,16)))* (1-ts_rank(returns,32)))',
    'alpha_036': '((2.21 * cs_rank(correlation(close - open, delay(volume, 1), 15)))+ (0.7 * cs_rank(open - close))+ (0.73 * cs_rank(ts_rank(delay(-1 * returns, 6), 5))) + cs_rank(abs(correlation(vwap, adv(vwap,volume,20), 6)))+ (0.6 * cs_rank(((ts_sum(close, 200) / 200) - open) * (close - open))))',
    'alpha_037': '(cs_rank(correlation(delay((open-close),1),close,200))+cs_rank((open-close)))',
    'alpha_038': '((-1 * cs_rank(ts_rank(close,10))) * cs_rank((close/open)))',
    'alpha_039': '((-1*cs_rank(delta(close,7)*(1-cs_rank(ts_decay_linear((volume/adv(vwap,volume,20)),9))))) * (1+cs_rank(ts_sum(returns,250))))',
    'alpha_040': '((-1*cs_rank(stddev(high,10)))*correlation(high, volume, 10))',
    'alpha_041': '(((high*low)**0.5)-vwap)',
    'alpha_042': '(cs_rank((vwap-close))/cs_rank((vwap+close)))',
    'alpha_043': '(ts_rank((volume / adv(vwap,volume,20)), 20) * ts_rank((-1 * delta(close, 7)), 8))',
    'alpha_044': '(-1 * correlation(high, cs_rank(volume), 5))',
    'alpha_045': '(-1 * ((cs_rank((ts_sum(delay(close,5),20)/20)))*correlation(close,volume,2)*cs_rank(correlation(ts_sum(close,5),ts_sum(close,20),2))))',
    'alpha_046': '(close*0-1).where((((delay(close,20) - delay(close,10)) / 10)- ((delay(close,10) - close) / 10)) > 0.25,(close*0 + 1)).where((((delay(close,20) - delay(close,10)) / 10)- ((delay(close,10) -close) / 10)) < 0,-1 * (close - delay(close,1)))',
    'alpha_047': '((((cs_rank((1 / close)) * volume) / adv(vwap,volume,20)) * ((high * cs_rank((high -close))) /(sum(high, 5) /5))) - cs_rank((vwap - delay(vwap, 5))))',
    # 048 目前無產業變數
    'alpha_048': '',
    'alpha_049': '(((((delay(close,20) - delay(close,10)) / 10) - ((delay(close,10) - close) / 10))).where(((((delay(close,20) - delay(close,10)) / 10) - ((delay(close,10) - close) / 10))) < -0.1, 1).where(((((delay(close,20) - delay(close,10)) / 10) - ((delay(close,10) - close) / 10))) >= -0.1, -1*(close - delay(close,1))))',
    'alpha_050': '(-1 * ts_max(cs_rank(correlation(cs_rank(volume), cs_rank(vwap), 5)), 5))',
    'alpha_051': '(((((delay(close,20) - delay(close,10)) / 10) - ((delay(close,10) - close) / 10))) .where(((((delay(close,20) - delay(close,10)) / 10) - ((delay(close,10) - close) / 10))) < -0.05, 1) .where(((((delay(close,20) - delay(close,10)) / 10) - ((delay(close,10) - close) / 10))) >= -0.05, -1*(close - delay(close,1))))',
    'alpha_052': ' ((((-1 * ts_min(low, 5)) + delay(ts_min(low, 5), 5)) * cs_rank(((ts_sum(returns, 240) -ts_sum(returns, 20)) / 220))) * ts_rank(volume, 5))',
    # 053 有inf 還沒調整
    'alpha_053': '(-1 * delta((((close - low) - (high - close)) / (close - low)), 9))',
    'alpha_054': '((-1 * ((low - close) * (open**5))) / ((low - high) * (close**5)))',
    'alpha_055': '(-1 * correlation(cs_rank(((close - ts_min(low, 12)) / (ts_max(high, 12) - ts_min(low,12)))), cs_rank(volume), 6))',
    # 目前 cap 用 vwap
    'alpha_056': ' (0 - (1 * (cs_rank((ts_sum(returns, 10) / ts_sum(ts_sum(returns, 2), 3))) * cs_rank((returns * vwap)))))',
    'alpha_057': '(0 - (1 * ((close - vwap) / ts_decay_linear(cs_rank(ts_argmax(close, 30)), 2))))',
    # 058 目前無產業變數
    'alpha_058': '',
    # 059 目前無產業變數
    'alpha_059': '',
    'alpha_060': '(0 - (1 * ((2 * scale(cs_rank(((((close - low) - (high - close)) / (high - low)) * volume)))) -scale(cs_rank(ts_argmax(close, 10))))))',
    'alpha_061': '(cs_rank((vwap - ts_min(vwap, 16))) < cs_rank(correlation(vwap, adv(vwap,volume,180), 17))).where((cs_rank((vwap - ts_min(vwap, 16))) < cs_rank(correlation(vwap, adv(vwap,volume,180), 17))), 1).where(~(cs_rank((vwap - ts_min(vwap, 16))) < cs_rank(correlation(vwap, adv(vwap,volume,180), 17))), 0)',
    'alpha_062': '((cs_rank(correlation(vwap, ts_sum(adv(vwap,volume,20), 22), 9)) < cs_rank(((cs_rank(open) +cs_rank(open)) < (cs_rank(((high + low) / 2)) + cs_rank(high))))) * -1)',
    # 063 目前無產業變數
    'alpha_063': '',
    'alpha_064': '((cs_rank(correlation(ts_sum(((open * 0.178404) + (low * (1 - 0.178404))), 12),ts_sum(adv(vwap,volume,120), 12), 16)) < cs_rank(delta(((((high + low) / 2) * 0.178404) + (vwap * (1 -0.178404))), 4))) * -1)',
    # 怪，重新計算
    'alpha_065': '',
    'alpha_066': '((cs_rank(ts_decay_linear(delta(vwap, 3), 7)) + ts_rank(ts_decay_linear(((((low* 0.96633) + (low * (1 - 0.96633))) - vwap) / (open - ((high + low) / 2))), 11), 6)) * -1)',
    # 067 目前無產業變數
    'alpha_067': '',
    'alpha_068': '((ts_rank(correlation(cs_rank(high), cs_rank(adv(vwap,volume,15)), 9), 14) <cs_rank(delta(((close * 0.518371) + (low * (1 - 0.518371))), 1))) * -1)',
    # 069 目前無產業變數
    'alpha_069': '',
    # 070 目前無產業變數
    'alpha_070': '',
    # 怪，重新計算
    'alpha_071': '',
    'alpha_072': '(cs_rank(ts_decay_linear(correlation(((high + low) / 2), adv(vwap,volume,40), 8), 10))/cs_rank(ts_decay_linear(correlation(ts_rank(vwap, 3),ts_rank(volume, 18), 6),2)))',
    # 怪，重新計算
    'alpha_073': '',
    'alpha_074': '((cs_rank(correlation(close, sum(adv(vwap,volume,30), 37), 15)) < cs_rank(correlation(cs_rank(((high * 0.0261661) + (vwap * (1 - 0.0261661)))), cs_rank(volume), 11)))* -1)',
    'alpha_075': '1 * (cs_rank(correlation(vwap, volume, 4)) < cs_rank(correlation(cs_rank(low), cs_rank(adv(vwap,volume,50)), 12)))',
    # 076 目前無產業變數
    'alpha_076': '',
    # 077 可能有誤
    'alpha_077': 'ts_min(cs_rank(ts_decay_linear(((((high + low) / 2) -vwap)), 20)), 5 )',
    'alpha_078': '(cs_rank(correlation(ts_sum(((low * 0.352233) + (vwap * (1 - 0.352233))), 19),ts_sum(adv(vwap,volume,40), 19), 6))**cs_rank(correlation(cs_rank(vwap), cs_rank(volume), 5)))',
    # 079 目前無產業變數
    'alpha_079': '',
    # 080 目前無產業變數
    'alpha_080': '',
    # 好像都是0?
    'alpha_081': '(cs_rank(log(ts_product(cs_rank((cs_rank(correlation(vwap, sum(adv(vwap,volume,10), 49), 8))**4)), 15))) < cs_rank(correlation(cs_rank(vwap), cs_rank(volume), 5))) * -1',
    # 082 目前無產業變數
    'alpha_082': '',
    'alpha_083': '((cs_rank(delay(((high - low) / (sum(close, 5) / 5)), 2)) * cs_rank(cs_rank(volume))) / (((high -low) / (sum(close, 5) / 5)) / (vwap - close)))',
    'alpha_084': 'signed_power(ts_rank((vwap - ts_max(vwap, 15)), 20), delta(close,4))',
    'alpha_085': '(cs_rank(correlation(((high * 0.876703) + (close * (1 - 0.876703))), adv(vwap,volume,30),9))**cs_rank(correlation(ts_rank(((high + low) / 2), 3), ts_rank(volume, 10),7)))',
    # 好像都是0?
    'alpha_086': '((ts_rank(correlation(close, sum(adv(vwap,volume,20), 14), 6), 20) < cs_rank(((open+ close) - (vwap + open)))) * -1)',
    # 087 目前無產業變數
    'alpha_087': '',
    'alpha_088': 'vmin(cs_rank(ts_decay_linear((cs_rank(open) + cs_rank(low) - cs_rank(high) - cs_rank(close)), 8)), ts_rank(ts_decay_linear(correlation(ts_rank(close, 8), ts_rank(adv(vwap,volume,60),20), 8), 6), 2) )',
    # 089 目前無產業變數
    'alpha_089': '',
    # 090 目前無產業變數
    'alpha_090': '',
    # 091 目前無產業變數
    'alpha_091': '',
    'alpha_092': 'vmin(ts_rank(ts_decay_linear(((((high + low) / 2) + close) < (low + open)), 14),18), ts_rank(ts_decay_linear(correlation(cs_rank(low), cs_rank(adv(vwap,volume,30)), 7), 6),6))',
    # 093 目前無產業變數
    'alpha_093': '',
    'alpha_094': '((cs_rank((vwap - ts_min(vwap, 11)))**ts_rank(correlation(ts_rank(vwap,19), ts_rank(adv(vwap,volume,60), 4), 18), 2)) * -1)',
    'alpha_095': '(close*0 + 1).where(cs_rank((open - ts_min(open, 12))) < ts_rank((cs_rank(correlation(ts_sum(((high + low)/2), 19), sum(adv(vwap, volume, 40), 19), 12))**5), 11),0)',
    'alpha_096': '(vmax(ts_rank(ts_decay_linear(correlation(cs_rank(vwap), cs_rank(volume), 3),4), 8), ts_rank(ts_decay_linear(ts_argmax(correlation(ts_rank(close, 7),ts_rank(adv(vwap,volume,60), 4), 3), 12), 14), 13)) * -1)',
    # 097 目前無產業變數
    'alpha_097': '',
    'alpha_098': '(cs_rank(ts_decay_linear(correlation(vwap, ts_sum(adv(vwap,volume,5), 26), 4), 7)) - cs_rank(ts_decay_linear(ts_rank(ts_argmin(correlation(cs_rank(open), cs_rank(adv(vwap,volume,15)), 20), 8),6), 8)))',
    'alpha_099': '((cs_rank(correlation(ts_sum(((high + low) / 2), 19), ts_sum(adv(vwap,volume,60), 19), 8)) < cs_rank(correlation(low, volume, 6))) * -1)',
    'alpha_100': '',
    'alpha_101': '((close - open) / ((high - low) + .001))'
    }

for name, expression in alpha101_expressions.items():
    print(f"計算 {name}...")
    try:
        df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
        print(f"  -> {name} 計算完成。")
    except Exception as e:
        print(f"  -> 計算 {name} 時發生錯誤: {e}")

計算 alpha_001...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/636097810.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  -> alpha_001 計算完成。
計算 alpha_002...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4026587022.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_series = temp_df.groupby('ticker').apply(rolling_corr).reset_index(level=0, drop=True)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/636097810.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  -> alpha_002 計算完成。
計算 alpha_003...


...

/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4026587022.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_series = temp_df.groupby('ticker').apply(rolling_corr).reset_index(level=0, drop=True)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4026587022.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_series = temp_df.groupby('ticker').apply(rolling_corr).reset_inde

  -> alpha_098 計算完成。
計算 alpha_099...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4026587022.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_series = temp_df.groupby('ticker').apply(rolling_corr).reset_index(level=0, drop=True)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4026587022.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_series = temp_df.groupby('ticker').apply(rolling_corr).reset_inde

  -> alpha_099 計算完成。
計算 alpha_100...
  -> 計算 alpha_100 時發生錯誤: invalid syntax (<string>, line 0)
計算 alpha_101...
  -> alpha_101 計算完成。


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/636097810.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/636097810.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


In [5]:
alpha158_expressions ={
    # kbar 因子（9)
    'kbar1': '(close-open)/open',
    'kbar2': '(high-low)/open',
    'kbar3': '(close-open)/((high-low)+1e-9)',
    'kbar4': '(high-vmax(open, close))/open',
    'kbar5': '(high-vmax(open, close))/((high-low)+1e-9)',
    'kbar6': '(vmin(open, close)-low)/open',
    'kbar7': '(vmin(open, close)-low)/((high-low)+1e-9)',
    'kbar8': '(2*close-high-low)/open',
    'kbar9': '(2*close-high-low)/((high-low)+1e-9)',

    # price 因子(4)
    'price1': 'open/close',
    'price2': 'high/close',
    'price3': 'low/close',
    'price4': 'vwap/close',

    # rolling 因子(29x5)
    # a. 趨勢類型(5x5)
    'roc5': '(delay(close, 5)/close)',
    'roc10': '(delay(close, 10)/close)',
    'roc20': '(delay(close, 20)/close)',
    'roc30': '(delay(close, 30)/close)',
    'roc60': '(delay(close, 60)/close)',

    'ma5': 'sma(close, 5)/close',
    'ma10': 'sma(close, 5)/close',
    'ma20': 'sma(close, 20)/close',
    'ma30': 'sma(close, 30)/close',
    'ma60': 'sma(close, 60)/close',

    'beta5': 'ts_slope(close, 5)/close',
    'beta10': 'ts_slope(close, 10)/close',
    'beta20': 'ts_slope(close, 20)/close',
    'beta30': 'ts_slope(close, 30)/close',
    'beta60': 'ts_slope(close, 60)/close',

    "rsq5": 'ts_rsq(close,5)',
    "rsq10": 'ts_rsq(close,10)',
    "rsq20": 'ts_rsq(close,20)',
    "rsq30": 'ts_rsq(close,30)',
    "rsq60": 'ts_rsq(close,60)',

    'resi5': 'ts_resi((close, 5)/close)',
    'resi10': 'ts_resi((close, 10)/close)',
    'resi20': 'ts_resi((close, 20)/close)',
    'resi30': 'ts_resi((close, 30)/close)',
    'resi60': 'ts_resi((close, 60)/close)',

    # b.波動類型(6x5)
    'std5': 'stddev(close,5)/close',
    'std10': 'stddev(close,10)/close',
    'std20': 'stddev(close,20)/close',
    'std30': 'stddev(close,30)/close',
    'std60': 'stddev(close,60)/close',

    'max5': 'ts_max(high, 5)/close',
    'max10': 'ts_max(high, 10)/close',
    'max20': 'ts_max(high, 20)/close',
    'max30': 'ts_max(high, 30)/close',
    'max60': 'ts_max(high, 60)/close',

    'min5': 'ts_min(low,5)/close',
    'min10': 'ts_min(low,10)/close',
    'min20': 'ts_min(low,20)/close',
    'min30': 'ts_min(low,30)/close',
    'min60': 'ts_min(low,60)/close',

    'qtlu5': 'ts_Quantile(close, 5, 0.8)/close',
    'qtlu10': 'ts_Quantile(close, 10, 0.8)/close',
    'qtlu20': 'ts_Quantile(close, 20, 0.8)/close',
    'qtlu30': 'ts_Quantile(close, 30, 0.8)/close',
    'qtlu60': 'ts_Quantile(close, 60, 0.8)/close',

    'qtld5': 'ts_Quantile(close, 5, 0.2)/close',
    'qtld10': 'ts_Quantile(close, 10, 0.2)/close',
    'qtld20': 'ts_Quantile(close, 20, 0.2)/close',
    'qtld30': 'ts_Quantile(close, 30, 0.2)/close',
    'qtld60': 'ts_Quantile(close, 60, 0.2)/close',

    'rsv5': '(close - ts_min(low, 5)) / (ts_max(high, 5) - ts_min(low, 5))',
    'rsv10': '(close - ts_min(low, 10)) / (ts_max(high, 10) - ts_min(low, 10))',
    'rsv20': '(close - ts_min(low, 20)) / (ts_max(high, 20) - ts_min(low, 20))',
    'rsv30': '(close - ts_min(low, 30)) / (ts_max(high, 30) - ts_min(low, 30))',
    'rsv60': '(close - ts_min(low, 60)) / (ts_max(high, 60) - ts_min(low, 60))',

    # c.時間週期類型(3x5)
    'imax5': 'IdxMax(high, 5) / 5',
    'imax10': 'IdxMax(high, 10) / 10',
    'imax20': 'IdxMax(high, 20) / 20',
    'imax30': 'IdxMax(high, 30) / 30',
    'imax60': 'IdxMax(high, 60) / 60',

    'imin5': 'IdxMin(low, 5) / 5',
    'imin10': 'IdxMin(low, 10) / 10',
    'imin20': 'IdxMin(low, 20) / 20',
    'imin30': 'IdxMin(low, 30) / 30',
    'imin60': 'IdxMin(low, 60) / 60',

    'imxd5': '(IdxMax(high, 5) - IdxMin(low, 5))/5',
    'imxd10': '(IdxMax(high, 10) - IdxMin(low, 10))/10',
    'imxd20': '(IdxMax(high, 20) - IdxMin(low, 20))/20',
    'imxd30': '(IdxMax(high, 30) - IdxMin(low, 30))/30',
    'imxd60': '(IdxMax(high, 60) - IdxMin(low, 60))/60',

    'vp_corr5': 'correlation(close, log(volume+1), 5)',
    'vp_corr10': 'correlation(close, log(volume+1), 10)',
    'vp_corr20': 'correlation(close, log(volume+1), 20)',
    'vp_corr30': 'correlation(close, log(volume+1), 30)',
    'vp_corr60': 'correlation(close, log(volume+1), 60)',

    'cord5': 'correlation(close/delay(close,1), log(volume/delay(volume,1)+1), 5)',
    'cord10': 'correlation(close/delay(close,1), log(volume/delay(volume,1)+1), 10)',
    'cord20': 'correlation(close/delay(close,1), log(volume/delay(volume,1)+1), 20)',
    'cord30': 'correlation(close/delay(close,1), log(volume/delay(volume,1)+1), 30)',
    'cord60': 'correlation(close/delay(close,1), log(volume/delay(volume,1)+1), 60)',

    'cntp5': 'sma(close>delay(close,1),5)',
    'cntp10': 'sma(close>delay(close,1),10)',
    'cntp20': 'sma(close>delay(close,1),20)',
    'cntp30': 'sma(close>delay(close,1),30)',
    'cntp60': 'sma(close>delay(close,1),60)',

    'cntn5': 'sma(close<delay(close,1),5)',
    'cntn10': 'sma(close<delay(close,1),10)',
    'cntn20': 'sma(close<delay(close,1),20)',
    'cntn30': 'sma(close<delay(close,1),30)',
    'cntn60': 'sma(close<delay(close,1),60)',

    'cntd5': 'sma(close>delay(close,1),5)-sma(close<delay(close,1),5)',
    'cntd10': 'sma(close>delay(close,1),10)-sma(close<delay(close,1),10)',
    'cntd20': 'sma(close>delay(close,1),20)-sma(close<delay(close,1),20)',
    'cntd30': 'sma(close>delay(close,1),30)-sma(close<delay(close,1),30)',
    'cntd60': 'sma(close>delay(close,1),60)-sma(close<delay(close,1),60)',

    'sump5': 'ts_sum(vmax(close-delay(close,1),0),5)/(ts_sum(abs(close-delay(close,1)),5))',
    'sump10': 'ts_sum(vmax(close-delay(close,1),0),10)/(ts_sum(abs(close-delay(close,1)),10))',
    'sump20': 'ts_sum(vmax(close-delay(close,1),0),20)/(ts_sum(abs(close-delay(close,1)),20))',
    'sump30': 'ts_sum(vmax(close-delay(close,1),0),30)/(ts_sum(abs(close-delay(close,1)),30))',
    'sump60': 'ts_sum(vmax(close-delay(close,1),0),60)/(ts_sum(abs(close-delay(close,1)),60))',

    'sumn5': 'ts_sum(vmax(delay(close,1)-close,0),5)/(ts_sum(abs(close-delay(close,1)),5))',
    'sumn10': 'ts_sum(vmax(delay(close,1)-close,0),10)/(ts_sum(abs(close-delay(close,1)),10))',
    'sumn20': 'ts_sum(vmax(delay(close,1)-close,0),20)/(ts_sum(abs(close-delay(close,1)),20))',
    'sumn30': 'ts_sum(vmax(delay(close,1)-close,0),30)/(ts_sum(abs(close-delay(close,1)),30))',
    'sumn60': 'ts_sum(vmax(delay(close,1)-close,0),60)/(ts_sum(abs(close-delay(close,1)),60))',

    'sumd5': '(ts_sum(vmax(close-delay(close,1),0),5)-ts_sum(vmax(delay(close,1)-close,0),5))/(ts_sum(abs(close-delay(close,1)),5))',
    'sumd10': '(ts_sum(vmax(close-delay(close,1),0),10)-ts_sum(vmax(delay(close,1)-close,0),10))/(ts_sum(abs(close-delay(close,1)),10))',
    'sumd20': '(ts_sum(vmax(close-delay(close,1),0),20)-ts_sum(vmax(delay(close,1)-close,0),20))/(ts_sum(abs(close-delay(close,1)),20))',
    'sumd30': '(ts_sum(vmax(close-delay(close,1),0),30)-ts_sum(vmax(delay(close,1)-close,0),30))/(ts_sum(abs(close-delay(close,1)),30))',
    'sumd60': '(ts_sum(vmax(close-delay(close,1),0),60)-ts_sum(vmax(delay(close,1)-close,0),60))/(ts_sum(abs(close-delay(close,1)),60))',

    # 成交量波動
    'vma5': 'sma(volume, 5)/(volume)',
    'vma10': 'sma(volume, 10)/(volume)',
    'vma20': 'sma(volume, 20)/(volume)',
    'vma30': 'sma(volume, 30)/(volume)',
    'vma60': 'sma(volume, 60)/(volume)',

    'vstd5': 'stddev(volume, 5)/(volume)',
    'vstd10': 'stddev(volume, 10)/(volume)',
    'vstd20': 'stddev(volume, 20)/(volume)',
    'vstd30': 'stddev(volume, 30)/(volume)',
    'vstd60': 'stddev(volume, 60)/(volume)',

    # 成交量加權類型（4x5）
    'wvma5': 'stddev(abs(close/delay(close,1)-1)*volume, 5)/(sma(abs(close/delay(close,1)-1)*volume, 5))',
    'wvma10': 'stddev(abs(close/delay(close,1)-1)*volume, 10)/(sma(abs(close/delay(close,1)-1)*volume, 10))',
    'wvma20': 'stddev(abs(close/delay(close,1)-1)*volume, 20)/(sma(abs(close/delay(close,1)-1)*volume, 20))',
    'wvma30': 'stddev(abs(close/delay(close,1)-1)*volume, 30)/(sma(abs(close/delay(close,1)-1)*volume, 30))',
    'wvma60': 'stddev(abs(close/delay(close,1)-1)*volume, 60)/(sma(abs(close/delay(close,1)-1)*volume, 60))',

    'vsump5': 'ts_sum(vmax(volume-delay(volume,1),0),5)/(ts_sum(abs(volume-delay(volume,1)),5))',
    'vsump10': 'ts_sum(vmax(volume-delay(volume,1),0),10)/(ts_sum(abs(volume-delay(volume,1)),10))',
    'vsump20': 'ts_sum(vmax(volume-delay(volume,1),0),20)/(ts_sum(abs(volume-delay(volume,1)),20))',
    'vsump30': 'ts_sum(vmax(volume-delay(volume,1),0),30)/(ts_sum(abs(volume-delay(volume,1)),30))',
    'vsump60': 'ts_sum(vmax(volume-delay(volume,1),0),60)/(ts_sum(abs(volume-delay(volume,1)),60))',

    'vsumn5': 'ts_sum(vmax(delay(volume,1)-volume,0),5)/(ts_sum(abs(volume-delay(volume,1)),5))',
    'vsumn10': 'ts_sum(vmax(delay(volume,1)-volume,0),10)/(ts_sum(abs(volume-delay(volume,1)),10))',
    'vsumn20': 'ts_sum(vmax(delay(volume,1)-volume,0),20)/(ts_sum(abs(volume-delay(volume,1)),20))',
    'vsumn30': 'ts_sum(vmax(delay(volume,1)-volume,0),30)/(ts_sum(abs(volume-delay(volume,1)),30))',
    'vsumn60': 'ts_sum(vmax(delay(volume,1)-volume,0),60)/(ts_sum(abs(volume-delay(volume,1)),60))',
    
    'vsumd5': '(ts_sum(vmax(volume-delay(volume,1),0),5)-ts_sum(vmax(delay(volume,1)-volume,0),5))/(ts_sum(abs(volume-delay(volume,1)),5))',
    'vsumd10': '(ts_sum(vmax(volume-delay(volume,1),0),10)-ts_sum(vmax(delay(volume,1)-volume,0),10))/(ts_sum(abs(volume-delay(volume,1)),10))',
    'vsumd20': '(ts_sum(vmax(volume-delay(volume,1),0),20)-ts_sum(vmax(delay(volume,1)-volume,0),20))/(ts_sum(abs(volume-delay(volume,1)),20))',
    'vsumd30': '(ts_sum(vmax(volume-delay(volume,1),0),30)-ts_sum(vmax(delay(volume,1)-volume,0),30))/(ts_sum(abs(volume-delay(volume,1)),30))',
    'vsumd60': '(ts_sum(vmax(volume-delay(volume,1),0),60)-ts_sum(vmax(delay(volume,1)-volume,0),60))/(ts_sum(abs(volume-delay(volume,1)),60))',
}

for name, expression in alpha158_expressions.items():
    print(f"計算 {name}...")
    try:
        df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
        print(f"  {name} 計算完成。")
    except Exception as e:
        print(f"  計算 {name} 時發生錯誤: {e}")

計算 kbar1...
  kbar1 計算完成。
計算 kbar2...
  kbar2 計算完成。
計算 kbar3...
  kbar3 計算完成。
計算 kbar4...
  kbar4 計算完成。
計算 kbar5...
  kbar5 計算完成。
計算 kbar6...
  kbar6 計算完成。
計算 kbar7...
  kbar7 計算完成。
計算 kbar8...
  kbar8 計算完成。
計算 kbar9...
  kbar9 計算完成。
計算 price1...
  price1 計算完成。
計算 price2...
  price2 計算完成。
計算 price3...
  price3 計算完成。
計算 price4...
  price4 計算完成。
計算 roc5...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4207368328.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4207368328.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4207368328.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usu

  roc5 計算完成。
計算 roc10...
  roc10 計算完成。
計算 roc20...
  roc20 計算完成。
計算 roc30...
  roc30 計算完成。
計算 roc60...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4207368328.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  roc60 計算完成。
計算 ma5...


...

/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4207368328.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  vsumd20 計算完成。
計算 vsumd30...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4207368328.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  vsumd30 計算完成。
計算 vsumd60...
  vsumd60 計算完成。


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/4207368328.py:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


In [6]:
revenue = df['Revenue'] # 營收
gross_profit = df['GrossProfit'] # 毛利
net_income = df['NetIncome'] # 淨利
total_equity=df['EquityAttributableToOwnersOfParent_x'] # 有x?
total_assets = df['TotalAssets'] # 總資產
total_liabilities = df['Liabilities']  # 總負債
currentliabilities = df['CurrentLiabilities']  # 流動負債
eps = df['EPS'] # 每股盈餘
interestexpense = df['InterestExpense'] # 利息費用

cashandcashequivalents = df['CashAndCashEquivalents']  # 現金及約當現金
accountsreceivablenet = df['AccountsReceivableNet']  # 應收帳款淨額

# EPS_ttm
df = df.sort_values(['ticker','report_date'])
df['eps_ttm'] = df.groupby('ticker')['EPS'].rolling(4, min_periods=4).sum().reset_index(level=0, drop=True)
eps_ttm = df['eps_ttm']

avg_total_assets = (
    df.groupby('ticker')['TotalAssets'].shift(0) +
    df.groupby('ticker')['TotalAssets'].shift(1)
) / 2

operatingIncome = df['OperatingIncome']  # 營業利益
incomeveforeincometax = df['IncomeBeforeIncomeTax']  # 稅前淨利
cashflowsfromoperatingactivities = df['CashFlowsFromOperatingActivities']  # 營業活動現金流量
depreciation = df['Depreciation']  # 折舊與攤銷
equityattributabletoownersofparent = df['EquityAttributableToOwnersOfParent_x']  # 股東權益

currentassets = df['CurrentAssets']  # 流動資產

#  有效稅率：TAX / 稅前淨利
pretax = incomeveforeincometax.replace(0, np.nan)
tax_rate = (df["TAX"] / pretax)

# 避免稅率亂飄：限制在 [0, 1]，且稅前 <= 0 時直接不算（NaN）
tax_rate = tax_rate.clip(lower=0, upper=1)
tax_rate = tax_rate.where(incomeveforeincometax > 0)

# NOPAT = EBIT * (1 - tax_rate) EBIT 用 OperatingIncome
df["NOPAT"] = operatingIncome * (1 - tax_rate)
NOPAT = df["NOPAT"]
# IC_C = TotalAssets - CashAndCashEquivalents - (CurrentLiabilities - ShorttermBorrowings)
df["IC_C"] = (
    df["TotalAssets"]
    - df["CashAndCashEquivalents"].fillna(0)
    - (df["CurrentLiabilities"].fillna(0) - df["ShorttermBorrowings"].fillna(0))
)
IC_C = df["IC_C"]

/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/3620144091.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['eps_ttm'] = df.groupby('ticker')['EPS'].rolling(4, min_periods=4).sum().reset_index(level=0, drop=True)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/3620144091.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["NOPAT"] = operatingIncome * (1 - tax_rate)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/3620144091.py:45: PerformanceWarning: DataFrame is highl

In [7]:
# 財務因子建立
finance_expressions ={
    'roe': 'net_income/total_equity',
    'roa': 'net_income/total_assets',
    'gross_profit_rate': 'gross_profit/revenue',
    'net_profit_rate': 'net_income/revenue',
    'ROIC': 'NOPAT / IC_C',
    'ep_ttm': 'eps_ttm/close',
    'ep_ttm_rank': 'cs_rank(eps_ttm/close)',
    'pb_ttm': '(close/eps)*(net_income/total_equity)',
    'pb_ttm_rank':'cs_rank((close/eps)*(net_income/total_equity))',
    'ps': '(close / eps)*(net_income / revenue)',
    'ps_rank': 'cs_rank((close / eps)*(net_income / revenue))',
    'debt_ratio': 'total_liabilities/total_assets',
    'ni_to_interest': 'net_income/interestexpense.where(interestexpense>0)',
    'asset_turnover': 'revenue/avg_total_assets',
    'asset_turnover_rank': 'cs_rank(revenue/avg_total_assets)',
    'op_margin':'operatingIncome / revenue',
    'pretax_margin':'incomeveforeincometax / revenue',
    'cfo_margin':'cashflowsfromoperatingactivities / revenue',
    'accruals':'(net_income - cashflowsfromoperatingactivities) / total_assets',
    'cash_conversion' : 'cashflowsfromoperatingactivities / net_income',
    'dep_rate' : 'depreciation / total_assets',
    'de_ratio' : 'total_liabilities / equityattributabletoownersofparent',
    'current_ratio' : 'currentassets / currentliabilities',
    'quick_ratio' :'(cashandcashequivalents + accountsreceivablenet) / currentliabilities',
    'cash_ratio' : 'cashandcashequivalents / currentliabilities',
    
    }

for name, expression in finance_expressions.items():
    print(f"計算 {name}...")
    try:
        df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
        print(f"  {name} 計算完成。")
    except Exception as e:
        print(f"  計算 {name} 時發生錯誤: {e}") 


計算 roe...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

  roe 計算完成。
計算 roa...
  roa 計算完成。
計算 gross_profit_rate...
  gross_profit_rate 計算完成。
計算 net_profit_rate...
  net_profit_rate 計算完成。
計算 ROIC...
  ROIC 計算完成。
計算 ep_ttm...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  ep_ttm 計算完成。
計算 ep_ttm_rank...


...

/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  dep_rate 計算完成。
計算 de_ratio...
  de_ratio 計算完成。
計算 current_ratio...


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)
/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


  current_ratio 計算完成。
計算 quick_ratio...
  quick_ratio 計算完成。
計算 cash_ratio...
  cash_ratio 計算完成。


/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_88321/733161481.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = eval(expression).replace([np.inf, -np.inf], np.nan)


In [8]:
df.to_pickle('data/factors_data_log60.pkl')